# VIX Wavelets

In [ ]:
!pip install yfinance pywt statsmodels pandas numpy matplotlib

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import pywt
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Step 1: Download VIX Data
vix = yf.download("^VIX", start="2010-01-01", end="2025-01-01")['Close']

# Step 2: Perform Wavelet Decomposition
wavelet = 'db4'  # Daubechies wavelet
level = 3  # Number of decomposition levels

coeffs = pywt.wavedec(vix, wavelet, level=level)
approximation = coeffs[0]  # Approximate part (low-frequency)
details = coeffs[1:]  # Detail parts (high-frequency)

# Step 3: Fit ARIMA model on the Approximate Component
train_size = int(len(approximation) * 0.8)
train, test = approximation[:train_size], approximation[train_size:]

model = ARIMA(train, order=(2,1,2))
model_fit = model.fit()

# Step 4: Forecast future values
forecast_steps = len(test)
forecast = model_fit.forecast(steps=forecast_steps)

# Step 5: Reconstruct the signal using inverse wavelet transform
new_coeffs = [np.concatenate([train, forecast])] + details
reconstructed_signal = pywt.waverec(new_coeffs, wavelet)

# Step 6: Plot Results
plt.figure(figsize=(10,5))
plt.plot(vix.index[-len(reconstructed_signal):], reconstructed_signal, label="Wavelet Forecast", linestyle="--")
plt.plot(vix.index, vix, label="Actual VIX", alpha=0.7)
plt.legend()
plt.title("VIX Forecast using Wavelet and ARIMA")
plt.show()
